# MS/MS hypothetical structure spectra template notebook
- this notebook takes ms/ms spectra and matches the observed ions to hypothetical ions generated from modified peptides
- you have to specify:
    - data location / names
    - peptide sequence
    - modifications present
    - MS spectra filtering parameters

# Notes

Take note of anything you like here. It'll be exported with the notebook when you run the notebook export.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import itertools
import glob
import os
import re
import matplotlib.pyplot as plt
import nbconvert

# Import the custom modules
from msms_structure_annot import plotters
from msms_structure_annot import msprocess
from msms_structure_annot import hsmakers
from msms_structure_annot import scoring

In [ ]:
# Import relevant paths to use
from msms_structure_annot.paths import data_dir
from msms_structure_annot.paths import reports_dir

## User-specified info

### Experiment / report name

In [ ]:
# Specify the name of this notebook (needed to export it in the end)
this_notebook_name = '*.ipynb'

# Specify msms files directory in the data_dir folder and load path to them
exp_name = '' # Parent directory name for reports and such
exp_dir = exp_name # Folder name where the data is (relative to the ./data directory)

# Change this if you want to generate different reports; It will currently overwrite any reports currently in there
report_id = 'report001'

exp_data_folder = data_dir / exp_dir
output_folder = reports_dir / exp_name / report_id

if not os.path.isdir(exp_data_folder):
    raise NotADirectoryError('Cannot find directory at provided data path!')

print('Data folder:\n', exp_data_folder, '\n')
print('Reports folder:\n', output_folder)

### Sequence and PTM info

In [ ]:
# Original AA sequence
parent_seq = 'XXXXXXXX' # Will be referred to as one-indexed from here on

# Define N and C-term modifications and their mass shifts
N_term_mod = 0 # polymerized AA masses already include proton
C_term_mod = 18.0027
proton_m = 1.0078
# Define number of charges to calculate m/z values for
charges = [1,2,3]

ptm_dict = {
    'name': [
        '',  # Name of the modification type
    ],
    'm_shift': [  # Mass shift for a given modification
        0, 
    ], 
    'num_mods': [ # Total number of modifications observed
        0, 
    ], 
    'poss_mod_pos': [ # Potential modification positions (one-indexed)
        [1,2,3],
    ],
    'type': [  # Type of modification (ring or point); (ring feature not currently implemented)
        'point',
    ]
}

In [ ]:
"""Examples of PTM specification:

# This is what single PTMs look like

ptm_dict = {
    'name': [
        'rSAM thioether',  # Name of the modification type
    ],
    'm_shift': [  # Mass shift for a given modification
        -1.007, 
    ], 
    'num_mods': [ # Total number of modifications observed
        2, 
    ], 
    'poss_mod_pos': [ # Potential modification positions (one-indexed)
        [18,22,15, 5],
    ],
    'type': [  # Type of modification (ring or point); (ring feature not currently implemented)
        'point',
    ]
}

# This is what multiple PTMs would look like

ptm_dict = {
    'name': [
        'rSAM thioether', 
        'dehydration'
    ],
    'm_shift': [  # Mass shift for a given modification
        -1.007, 
        -16.00
    ], 
    'num_mods': [ # Total number of modifications observed
        2, 
        2
    ], 
    'poss_mod_pos': [ # Potential modification positions (one-indexed)
        [18,22,15],
        [2,5,7] 
    ] 
}

"""

### MS spectra processing parameters

In [ ]:
## User-specified parameters
tol = 0.05 # mass-deviation tolerance; Default 0.4
sn_thr = 0.1 # signal-to-noise threshold (must be sn_thr times above background for ion to count); Default 5
N = 12800 # Number of sections to split m/z datapoints into when calculating background values; Default 500
upper_lim = 10 # limit to deviance above average ms ion intensity to set ion values to a limit; Default 50

## Importing

### PTMs

In [ ]:
# Import PTMs into a dataframe
ptms_df = pd.DataFrame(ptm_dict)
ptms_df.index.name = 'ptm_id'
ptms_df = ptms_df.reset_index()
ptms_df

### Mass spec files

In [ ]:
ms_df = msprocess.import_ms_files(exp_data_folder)

## Processing mass spectra files

- cap the maximum value of high abundance ions
- calculate background level and filter by signal to noise

In [ ]:
# Make sure everything is sorted before running background calculations
ms_df.sort_values(by = ['spec_num', 'm/z'], inplace= True)

# First apply abundance ceiling to each spectra file
ms_df['abund_ceil'] =  ms_df.groupby('spec_num')['orig_abundance'].apply(msprocess.abund_ceiling, upper_lim)
# Then calculate the background signal within each section
ms_df['bkgd'] = ms_df.groupby('spec_num')['abund_ceil'].transform(msprocess.bkgd_calc_ser, N)

In [ ]:
ms_df.head()

In [ ]:
# Filter masses based off how far above the background signal they are according to the signal to noise threshold
ms_df_sn_filter = ms_df[ms_df['abund_ceil'] > sn_thr*ms_df['bkgd']].reset_index(drop = True)

## Create all hypothetical structures (hs) and fragment them

In [ ]:
# Generate all possible hypothetical structures from provided PTM dataframe
hs_df = hsmakers.gen_hss(ptms_df)
hs_df

### Fragment and assign masses

In [ ]:
frag_df = hsmakers.frag_hs(hs_df, ptms_df, parent_seq, N_term_mod, C_term_mod)
frag_df.head(10)

### Add multiply-charged ions to hypothetical ion dataframe

- Look for multiply-charged ions instead of just single deconvoluted data:
    - which ones to look for:
        - M+H
        - M + 2H / 2
        - M + 3H / 3

In [ ]:
frag_df_charged = hsmakers.mk_charge_df(frag_df, charges, proton_m)
frag_df_charged.tail(10)

## Scoring hypothetical structures

### Match observed ions to hypothetical ions

In [ ]:
matched_df = scoring.match_ions(ms_df_sn_filter, frag_df_charged, tol)
matched_df.head()

### Use various scoring metrics to score the hypothetical structures

In [ ]:
# Create an empty score dataframe
scores_df = pd.DataFrame({'hs_id': [], 'score': [], 'score_method': []})
scores_df['hs_id'] = scores_df['hs_id'].astype(int)

# Iterate through scoring methods and append scored hypothetical structures
N_spec = len(ms_df['spec_num'].unique()) # calculate the number of ms/ms spectra used

for method in ['frac', 'weights']:
    scores_df = scores_df.append(scoring.score_wrapper(matched_df, frag_df_charged, N_spec, score_method = method))

# Sort it
scores_df = scores_df.sort_values(['score_method', 'score'], ascending = False)
scores_df

## Plots

### Barplots for scored hypothetical structures

In [ ]:
import seaborn as sns

#### Plot all metrics

In [ ]:
all_scores_fig = sns.FacetGrid(scores_df, row = 'score_method')
all_scores_fig.map(sns.barplot, 'hs_id', 'score', color = 'gainsboro', edgecolor = 'black')

#### Plot just a single metric

In [ ]:
## Can also plot with single metric and order it
metric = 'frac'

In [ ]:

sub_df = scores_df[scores_df['score_method'] == metric]
order = sub_df.sort_values(['score'], ascending = False)['hs_id'].values

single_scores_fig, ax = plt.subplots()
sns.barplot(x = 'hs_id', y = 'score', order = order, data = sub_df,
    color = 'gainsboro', edgecolor = 'black', ax = ax)
ax.set_xlabel('Hypothetical structure ID')
ax.set_ylabel('Score')
plt.show()

### Annotated mass spectra plots

In [ ]:
# Pick a single hypothetical structure to plot the matched ions for
hs_id_plot = 0
ms_file_nums = ms_df['spec_num'].unique()
print('Num spectra: ',len(ms_file_nums))

In [ ]:
# Plot formatting if you wish; Specify either a global set of limits (one tuple in a list)
#    or limits for each (one tuple for each spectra plot)
auto_yscale= True
xlims = [(0,2000)]
ylims = [(0,1e5)]
# Define a summed abudance lower limit for plotting hypothetical ion annotations
annot_sn_lim = 0

spectra_fig, axs = plotters.label_spectra_plot(ms_df, matched_df, ms_file_nums, hs_id = hs_id_plot,
                                                auto_yscale= auto_yscale, xlims = xlims, ylims = ylims,
                                                annot_sn_lim = annot_sn_lim)
plt.show()

### Combined mass spectra plots
- sum peaks across all spectra to increase S/N
- then annotate with a given hypothetical structure and make a single plot

#### Process
- must first round m/z values to be fewer significant figures than the machine error across multiple spectra

In [ ]:
## Specify number of decimal places to round to compensate for machine error
num_dec = 2

ms_df_round = ms_df.copy()
ms_df_round['m/z'] = ms_df_round['m/z'].round(decimals = num_dec)
ms_df_round.head()

In [ ]:
# Sum all instances of a given mass
combined_ms_df = ms_df_round.groupby(['m/z'])['orig_abundance'].sum().reset_index()
combined_ms_df.loc[:,'spec_num'] = 0 # Add spec num column for backward compatibility

# Match those masses to hypothetical masses
comb_matched_df = scoring.match_ions(combined_ms_df,frag_df_charged, tol = tol)
comb_matched_df.loc[:,'spec_num'] = 0 # Add spec num column for backward compatibility
comb_matched_df.head()

Plot it

In [ ]:
# Pick a single hypothetical structure to plot the matched ions for
hs_id_plot = 0
# Define a summed abudance lower limit for plotting hypothetical ion annotations
annot_sn_lim = 5000

comb_spectra_fig, axs = plotters.label_spectra_plot(
    combined_ms_df, comb_matched_df, ms_file_nums = [0], hs_id = hs_id_plot, 
    auto_yscale= auto_yscale, xlims = xlims, ylims = ylims, 
    annot_sn_lim = annot_sn_lim
)

plt.show()

# Exports to reports folder

NOTE: When you're ready to export the reports, it's best practice to rerun all the code cells above before exporting. This makes sure whatever we export down below is in the expected state.

Do this before exporting:
- Rerun all the cells above this point
- Save the Jupyter notebook

In [ ]:
## Make output directory if it doesn't already exist
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

# Output folder specified at the beginning of the notebook
print("Output folder:\n", output_folder)

### Excel file outputs

In [ ]:
# Export matched ions
matched_df.sort_values(['hs_id', 'spec_num', 'm/z'])
matched_df.to_excel(output_folder / 'matched_ions.xlsx', index = False)

# Export sorted score dataframe
scores_df.to_excel(output_folder / 'hs_scores.xlsx', index = False)
# Export hypothetical structures and their PTM locations
hs_df.to_excel(output_folder / 'hs_df.xlsx', index = True)
# Export PTM information
ptms_df.to_excel(output_folder / 'ptms_df.xlsx', index = False)


### Plots

In [ ]:
# Make the spectra plots output folder
if not os.path.isdir(output_folder / 'spectra_plots'):
    os.makedirs(output_folder / 'spectra_plots')

In [ ]:
all_scores_fig.savefig(output_folder / "all_scores_fig.pdf", dpi=300)
single_scores_fig.savefig(output_folder / "single_score_fig.pdf", dpi=300)
spectra_fig.savefig(output_folder / 'spectra_plots' / 'hs{}_matched_ions.pdf'.format(hs_id_plot), dpi=300)
comb_spectra_fig.savefig(
    output_folder / 'spectra_plots' / 'hs{}_summed_spectra_matches.pdf'.format(hs_id_plot), dpi=300)

In [ ]:
# If you want to export all the different hs ion matching spectra, iterate over this:
"""
ms_file_nums = ms_df['spec_num'].unique()

for hs_id_plot in hs_df['hs_id'].unique():
    spectra_fig, axs = plotters.label_spectra_plot(ms_df, matched_df, ms_file_nums, hs_id = hs_id_plot)
    spectra_fig.savefig(output_folder / 'spectra_plots' / 'hs{}_matched_ions.pdf'.format(hs_id_plot), dpi=300)
    comb_spectra_fig.savefig(
        output_folder / 'spectra_plots' / 'hs{}_summed_spectra_matches.pdf'.format(hs_id_plot), dpi=300)
    plt.close(spectra_fig)
"""

### The Jupyter notebook itself

Pretty hacky, but you have to put the name of the notebook (done at the top) to be able to export it (very hard to extract programmatically). This will save this notebook as an HTML file so you have a record of what happened.

In [ ]:
from nbconvert import HTMLExporter
import codecs
import nbformat

notebook_name = this_notebook_name
output_file_name = output_folder / str(exp_name + '_' + report_id + '.html')

exporter = HTMLExporter()
output_notebook = nbformat.read(notebook_name, as_version=4)

output, resources = exporter.from_notebook_node(output_notebook)
codecs.open(output_file_name, 'w', encoding='utf-8').write(output)